In [ ]:
import sympy as sp
from sympy import *  # noqa
from sympy.solvers.solveset import linsolve

sp.init_session()
sp.init_printing()

### Global variables:

In [ ]:
# Dimensional parameters
dd = 9
DD = 12

# Speed of light, time, time step
c = sp.symbols(r"c", real=True, positive=True)
t = sp.symbols(r"t", real=True)
tn = sp.symbols(r"t_n", real=True)
dt = sp.symbols(r"\Delta{t}", real=True, positive=True)

# Components of k vector, omega, norm of k vector
kx = sp.symbols(r"k_x", real=True)
ky = sp.symbols(r"k_y", real=True)
kz = sp.symbols(r"k_z", real=True)
om = sp.symbols(r"omega", real=True, positive=True)
knorm = sp.sqrt(kx**2 + ky**2 + kz**2)

### Method:
The goal is to solve a system of second-order ordinary differential equations in time of the form $\boldsymbol{\ddot{X}} = M \boldsymbol{X}$, where the $d \times d$ matrix $M$ has eigenpairs $(\lambda_i, \{\boldsymbol{v}_{i,1}, \dots, \boldsymbol{v}_{i,\mu_i}\})$, where $\mu_i$ denotes the algebraic multiplicity of $\lambda_i$ and $\sum_i \mu_i = d$.
Assuming that all eigenvalues are either zero or negative, we can write $\lambda_i = - \omega_i^2$, with $\omega_i = \sqrt{- \lambda_i} \geq 0$.
Then the solution of $\boldsymbol{\ddot{X}} = M \boldsymbol{X}$ reads
$$
\boldsymbol{X} = \sum_i \sum_j^{\mu_i} \left(a_{i,j} C(\omega_i, t) + b_{i,j} S(\omega_i, t) \right) \boldsymbol{v}_{i,j} \,,
$$
where $a_{i,j}$ and $b_{i,j}$ are integration constants to be determined by the initial conditions $\boldsymbol{X}(t_n)$ and $\boldsymbol{\dot{X}}(t_n)$, $C(\omega, t) = \cos(\omega \, (t - t_n))$, and
$$
S(\omega, t) =
\begin{cases}
(t - t_n) & \omega = 0 \,, \\
\dfrac{\sin(\omega \, (t - t_n))}{\omega} & \omega \neq 0 \,.
\end{cases}
$$
We remark that
$$
\begin{aligned}
& \boldsymbol{X}(t_n) = \sum_i \sum_j^{\mu_i} \left(a_{i,j} C(\omega_i, t_n) + b_{i,j} S(\omega_i, t_n) \right) \boldsymbol{v}_{i,j} = \sum_i \sum_j^{\mu_i} a_{i,j} \boldsymbol{v}_{i,j} \,, \\
& \boldsymbol{\dot{X}}(t_n) = \sum_i \sum_j^{\mu_i} \left(a_{i,j} \dot{C}(\omega_i, t_n) + b_{i,j} \dot{S}(\omega_i, t_n) \right) \boldsymbol{v}_{i,j} = \sum_i \sum_j^{\mu_i} b_{i,j} \boldsymbol{v}_{i,j} \,.
\end{aligned}
$$
In fact, the second time derivative of $\boldsymbol{X}$ yields
$$
\begin{split}
\boldsymbol{\ddot{X}} & = \sum_i \sum_j^{\mu_i} \left(a_{i,j} \ddot{C}(\omega_i, t) + b_{i,j} \ddot{S}(\omega_i, t) \right) \boldsymbol{v}_{i,j} \\
& = \sum_i (-\omega_i^2) \sum_j^{\mu_i} \left(a_{i,j} C(\omega_i, t) + b_{i,j} S(\omega_i, t) \right) \boldsymbol{v}_{i,j} = M \boldsymbol{X} \,,
\end{split}
$$
where we used the fact that, by definition, $M \boldsymbol{v}_{i,j} = \lambda_i \boldsymbol{v}_{i,j} = - \omega_i^2 \boldsymbol{v}_{i,j}$ for all $j = 1, \dots, \mu_i$.

### Auxiliary functions:

In [ ]:
def C(omega, t):
    return sp.cos(omega * (t - tn))


def S(omega, t):
    return (t - tn) if omega == 0.0 else sp.sin(omega * (t - tn)) / omega


def Xt(eigenpairs, a, b, t):
    """
    Compute X(t) according to the formulas above
    for a given set of eigenpairs and coefficients.
    """
    XX = zeros(DD, 1)
    # Index used for the integration constants a_n and b_n
    i = 0
    # Loop over matrix eigenpairs
    for ep in eigenpairs:
        # ep[0] is an eigenvalue and om = sp.sqrt(-ep[0])
        omega = 0.0 if ep[0] == 0.0 else om
        # am is the algebraic multiplicity of the eigenvalue
        am = ep[1]
        # vF is the list of all eigenvectors corresponding to the eigenvalue
        vX = ep[2]
        # Loop over algebraic multiplicity of the eigenvalue
        for j in range(am):
            XX += (a[i] * C(omega, t) + b[i] * S(omega, t)) * vX[j]
            i += 1
    return XX


def evolve(X, dX_dt, d2X_dt2):
    """
    Solve ordinary differential equation X'' = M*X.
    """
    # Set matrix for given ODE
    MX = zeros(DD)
    for i in range(DD):
        for j in range(DD):
            MX[i, j] = d2X_dt2[i].coeff(X[j], 1)
    # MX /= c**2

    print()
    print(r"Matrix:")
    display(MX)

    # Characteristic matrix
    lamda = sp.symbols(r"lamda")
    Id = eye(DD)
    MX_charmat = MX - lamda * Id

    # Characteristic polynomial
    MX_charpoly = MX_charmat.det()
    MX_charpoly = factor(MX_charpoly.as_expr())

    print(r"Characteristic polynomial:")
    display(MX_charpoly)

    MX_eigenvals = sp.solve(MX_charpoly, lamda)

    # List of eigenvectors
    MX_eigenvects = []

    # List of eigenpairs
    MX_eigenpairs = []

    # Compute eigenvectors as null spaces
    for ev in MX_eigenvals:
        # M - lamda * Id
        A = MX_charmat.subs(lamda, ev)
        A.simplify()

        print(r"Eigenvalue:")
        display(ev)

        print(r"Characteristic matrix:")
        display(A)

        # Perform Gaussian elimination (necessary for lamda != 0)
        if ev != 0.0:
            print(r"Gaussian elimination:")
            print(r"A[0,:] += A[1,:]")
            A[0, :] += A[1, :]
            print(r"A[0,:] += A[2,:]")
            A[0, :] += A[2, :]
            print(r"Swap A[0,:] and A[11,:]")
            row = A[11, :]
            A[11, :] = A[0, :]
            A[0, :] = row
            print(r"A[3,:] += A[4,:]")
            A[3, :] += A[4, :]
            print(r"A[3,:] += A[5,:]")
            A[3, :] += A[5, :]
            print(r"Swap A[3,:] and A[10,:]")
            row = A[10, :]
            A[10, :] = A[3, :]
            A[3, :] = row
            print(r"A[0,:] += A[3,:]")
            A[0, :] += A[3, :]
            print(r"A[0,:] += A[9,:]")
            A[0, :] += A[9, :]
            print(r"Swap A[0,:] and A[9,:]")
            row = A[9, :]
            A[9, :] = A[0, :]
            A[0, :] = row
            print(r"A[6,:] += A[8,:]")
            A[6, :] += A[8, :]
            print(r"A[6,:] += A[7,:]")
            A[6, :] += A[7, :]
            print(r"Swap A[6,:] and A[8,:]")
            row = A[8, :]
            A[8, :] = A[6, :]
            A[6, :] = row
            print(r"A[6,:] += A[7,:]")
            A[6, :] += A[7, :]
            print(r"A[4,:] += A[5,:]")
            A[4, :] += A[5, :]
            A.simplify()
            display(A)

        # Compute null space and store eigenvectors
        v = A.nullspace()
        MX_eigenvects.append(v)

        print(r"Eigenvectors:")
        display(v)

        # Store eigenpairs (eigenvalue, algebraic multiplicity, eigenvectors)
        MX_eigenpairs.append((ev, len(v), v))

        # print(r'Eigenpairs:')
        # display(MX_eigenpairs)

    # Verify that the eigenpairs satisfy the characteristic equations
    for ep in MX_eigenpairs:
        for j in range(ep[1]):
            diff = MX * ep[2][j] - ep[0] * ep[2][j]
            diff.simplify()
            if diff != zeros(DD, 1):
                print("The charcteristic equation is not verified for some eigenpairs")
                display(diff)

    # Define integration constants
    a = []
    b = []
    for i in range(DD):
        an = r"a_{:d}".format(i + 1)
        bn = r"b_{:d}".format(i + 1)
        a.append(sp.symbols(an))
        b.append(sp.symbols(bn))

    # Set equations corresponding to initial conditions
    lhs_a = Xt(MX_eigenpairs, a, b, tn) - X
    lhs_b = Xt(MX_eigenpairs, a, b, t).diff(t).subs(t, tn) - dX_dt

    # Compute integration constants from initial conditions
    # (convert list of tuples to list using list comprehension)
    a = list(linsolve(list(lhs_a), a))
    a = [item for el in a for item in el]
    b = list(linsolve(list(lhs_b), b))
    b = [item for el in b for item in el]

    # Evaluate solution at t = tn + dt
    X_new = Xt(MX_eigenpairs, a, b, tn + dt).expand()
    for d in range(DD):
        for Eij in E:
            X_new[d] = X_new[d].collect(Eij)
        for Bij in B:
            X_new[d] = X_new[d].collect(Bij)
        for Fi in F:
            X_new[d] = X_new[d].collect(Fi)
        for Gi in G:
            X_new[d] = X_new[d].collect(Gi)

    # Check correctness by taking *second* derivative
    # and comparing with initial right-hand side at time tn
    X_t = Xt(MX_eigenpairs, a, b, t)
    diff = X_t.diff(t).diff(t).subs(t, tn).subs(om, c * knorm).expand() - d2X_dt2
    diff.simplify()
    if diff != zeros(DD, 1):
        print("Integration in time failed")
        display(diff)

    return X_t, X_new

### First-order and second-order ODEs for $\boldsymbol{E}$, $\boldsymbol{B}$, $F$ and $G$:
Equations for the $\boldsymbol{E}$ field:
$$
\begin{alignat*}{3}
&  \frac{\partial E_{xx}}{\partial t} = c^2 i k_x (F_x + F_y  + F_z) \quad
&& \frac{\partial E_{xy}}{\partial t} = c^2 i k_y (B_{zx} + B_{zy} + B_{zz}) \quad
&& \frac{\partial E_{xz}}{\partial t} = -c^2 i k_z (B_{yx} + B_{yy} + B_{yz}) \\[5pt]
&  \frac{\partial E_{yx}}{\partial t} = -c^2 i k_x (B_{zx} + B_{zy} + B_{zz}) \quad
&& \frac{\partial E_{yy}}{\partial t} = c^2 i k_y (F_x + F_y + F_z) \quad
&& \frac{\partial E_{yz}}{\partial t} = c^2 i k_z (B_{xx} + B_{xy} + B_{xz}) \\[5pt]
&  \frac{\partial E_{zx}}{\partial t} = c^2 i k_x (B_{yx} + B_{yy} + B_{yz}) \quad
&& \frac{\partial E_{zy}}{\partial t} = -c^2 i k_y (B_{xx} + B_{xy} + B_{xz})\quad
&& \frac{\partial E_{zz}}{\partial t} = c^2 i k_z (F_x + F_y + F_z)
\end{alignat*}
$$

Equations for the $\boldsymbol{B}$ field:
$$
\begin{alignat*}{3}
&  \frac{\partial B_{xx}}{\partial t} = i k_x (G_x + G_y + G_z) \quad
&& \frac{\partial B_{xy}}{\partial t} = -i k_y (E_{zx} + E_{zy} + E_{zz}) \quad
&& \frac{\partial B_{xz}}{\partial t} = i k_z (E_{yx} + E_{yy} + E_{yz}) \\[5pt]
&  \frac{\partial B_{yx}}{\partial t} = i k_x (E_{zx} + E_{zy} + E_{zz}) \quad
&& \frac{\partial B_{yy}}{\partial t} = i k_y (G_x + G_y + G_z) \quad
&& \frac{\partial B_{yz}}{\partial t} = -i k_z (E_{xx} + E_{xy} + E_{xz}) \\[5pt]
&  \frac{\partial B_{zx}}{\partial t} = -i k_x (E_{yx} + E_{yy} + E_{yz}) \quad
&& \frac{\partial B_{zy}}{\partial t} = i k_y (E_{xx} + E_{xy} + E_{xz}) \quad
&& \frac{\partial B_{zz}}{\partial t} = i k_z (G_x + G_y + G_z)
\end{alignat*}
$$

Equations for the $F$ field:
$$
\frac{\partial F_x}{\partial t} = i k_x (E_{xx} + E_{xy} + E_{xz}) \quad
\frac{\partial F_y}{\partial t} = i k_y (E_{yx} + E_{yy} + E_{yz}) \quad
\frac{\partial F_z}{\partial t} = i k_z (E_{zx} + E_{zy} + E_{zz})
$$

Equations for the $G$ field:
$$
\frac{\partial G_x}{\partial t} = c^2 i k_x (B_{xx} + B_{xy} + B_{xz}) \quad
\frac{\partial G_y}{\partial t} = c^2 i k_y (B_{yx} + B_{yy} + B_{yz}) \quad
\frac{\partial G_z}{\partial t} = c^2 i k_z (B_{zx} + B_{zy} + B_{zz})
$$

In [ ]:
# indices  0     1     2     3     4     5     6     7     8
labels = ["xx", "xy", "xz", "yx", "yy", "yz", "zx", "zy", "zz"]

# E fields
Exx = sp.symbols(r"E_{xx}")
Exy = sp.symbols(r"E_{xy}")
Exz = sp.symbols(r"E_{xz}")
Eyx = sp.symbols(r"E_{yx}")
Eyy = sp.symbols(r"E_{yy}")
Eyz = sp.symbols(r"E_{yz}")
Ezx = sp.symbols(r"E_{zx}")
Ezy = sp.symbols(r"E_{zy}")
Ezz = sp.symbols(r"E_{zz}")
E = Matrix([[Exx], [Exy], [Exz], [Eyx], [Eyy], [Eyz], [Ezx], [Ezy], [Ezz]])

# B fields
Bxx = sp.symbols(r"B_{xx}")
Bxy = sp.symbols(r"B_{xy}")
Bxz = sp.symbols(r"B_{xz}")
Byx = sp.symbols(r"B_{yx}")
Byy = sp.symbols(r"B_{yy}")
Byz = sp.symbols(r"B_{yz}")
Bzx = sp.symbols(r"B_{zx}")
Bzy = sp.symbols(r"B_{zy}")
Bzz = sp.symbols(r"B_{zz}")
B = Matrix([[Bxx], [Bxy], [Bxz], [Byx], [Byy], [Byz], [Bzx], [Bzy], [Bzz]])

# F fields
Fx = sp.symbols(r"F_{x}")
Fy = sp.symbols(r"F_{y}")
Fz = sp.symbols(r"F_{z}")
F = Matrix([[Fx], [Fy], [Fz]])

# G fields
Gx = sp.symbols(r"G_{x}")
Gy = sp.symbols(r"G_{y}")
Gz = sp.symbols(r"G_{z}")
G = Matrix([[Gx], [Gy], [Gz]])

# dE/dt
dExx_dt = c**2 * I * kx * (Fx + Fy + Fz)
dExy_dt = c**2 * I * ky * (Bzx + Bzy + Bzz)
dExz_dt = -(c**2) * I * kz * (Byx + Byy + Byz)
dEyx_dt = -(c**2) * I * kx * (Bzx + Bzy + Bzz)
dEyy_dt = c**2 * I * ky * (Fx + Fy + Fz)
dEyz_dt = c**2 * I * kz * (Bxx + Bxy + Bxz)
dEzx_dt = c**2 * I * kx * (Byx + Byy + Byz)
dEzy_dt = -(c**2) * I * ky * (Bxx + Bxy + Bxz)
dEzz_dt = c**2 * I * kz * (Fx + Fy + Fz)
dE_dt = Matrix(
    [
        [dExx_dt],
        [dExy_dt],
        [dExz_dt],
        [dEyx_dt],
        [dEyy_dt],
        [dEyz_dt],
        [dEzx_dt],
        [dEzy_dt],
        [dEzz_dt],
    ]
)

# dB/dt
dBxx_dt = I * kx * (Gx + Gy + Gz)
dBxy_dt = -I * ky * (Ezx + Ezy + Ezz)
dBxz_dt = I * kz * (Eyx + Eyy + Eyz)
dByx_dt = I * kx * (Ezx + Ezy + Ezz)
dByy_dt = I * ky * (Gx + Gy + Gz)
dByz_dt = -I * kz * (Exx + Exy + Exz)
dBzx_dt = -I * kx * (Eyx + Eyy + Eyz)
dBzy_dt = I * ky * (Exx + Exy + Exz)
dBzz_dt = I * kz * (Gx + Gy + Gz)
dB_dt = Matrix(
    [
        [dBxx_dt],
        [dBxy_dt],
        [dBxz_dt],
        [dByx_dt],
        [dByy_dt],
        [dByz_dt],
        [dBzx_dt],
        [dBzy_dt],
        [dBzz_dt],
    ]
)

# dF/dt
dFx_dt = I * kx * (Exx + Exy + Exz)
dFy_dt = I * ky * (Eyx + Eyy + Eyz)
dFz_dt = I * kz * (Ezx + Ezy + Ezz)
dF_dt = Matrix([[dFx_dt], [dFy_dt], [dFz_dt]])

# dG/dt
dGx_dt = c**2 * I * kx * (Bxx + Bxy + Bxz)
dGy_dt = c**2 * I * ky * (Byx + Byy + Byz)
dGz_dt = c**2 * I * kz * (Bzx + Bzy + Bzz)
dG_dt = Matrix([[dGx_dt], [dGy_dt], [dGz_dt]])

# d2E/dt2
d2Exx_dt2 = c**2 * I * kx * (dFx_dt + dFy_dt + dFz_dt)
d2Exy_dt2 = c**2 * I * ky * (dBzx_dt + dBzy_dt + dBzz_dt)
d2Exz_dt2 = -(c**2) * I * kz * (dByx_dt + dByy_dt + dByz_dt)
d2Eyx_dt2 = -(c**2) * I * kx * (dBzx_dt + dBzy_dt + dBzz_dt)
d2Eyy_dt2 = c**2 * I * ky * (dFx_dt + dFy_dt + dFz_dt)
d2Eyz_dt2 = c**2 * I * kz * (dBxx_dt + dBxy_dt + dBxz_dt)
d2Ezx_dt2 = c**2 * I * kx * (dByx_dt + dByy_dt + dByz_dt)
d2Ezy_dt2 = -(c**2) * I * ky * (dBxx_dt + dBxy_dt + dBxz_dt)
d2Ezz_dt2 = c**2 * I * kz * (dFx_dt + dFy_dt + dFz_dt)
d2E_dt2 = Matrix(
    [
        [d2Exx_dt2],
        [d2Exy_dt2],
        [d2Exz_dt2],
        [d2Eyx_dt2],
        [d2Eyy_dt2],
        [d2Eyz_dt2],
        [d2Ezx_dt2],
        [d2Ezy_dt2],
        [d2Ezz_dt2],
    ]
)

# d2B/dt2
d2Bxx_dt2 = I * kx * (dGx_dt + dGy_dt + dGz_dt)
d2Bxy_dt2 = -I * ky * (dEzx_dt + dEzy_dt + dEzz_dt)
d2Bxz_dt2 = I * kz * (dEyx_dt + dEyy_dt + dEyz_dt)
d2Byx_dt2 = I * kx * (dEzx_dt + dEzy_dt + dEzz_dt)
d2Byy_dt2 = I * ky * (dGx_dt + dGy_dt + dGz_dt)
d2Byz_dt2 = -I * kz * (dExx_dt + dExy_dt + dExz_dt)
d2Bzx_dt2 = -I * kx * (dEyx_dt + dEyy_dt + dEyz_dt)
d2Bzy_dt2 = I * ky * (dExx_dt + dExy_dt + dExz_dt)
d2Bzz_dt2 = I * kz * (dGx_dt + dGy_dt + dGz_dt)
d2B_dt2 = Matrix(
    [
        [d2Bxx_dt2],
        [d2Bxy_dt2],
        [d2Bxz_dt2],
        [d2Byx_dt2],
        [d2Byy_dt2],
        [d2Byz_dt2],
        [d2Bzx_dt2],
        [d2Bzy_dt2],
        [d2Bzz_dt2],
    ]
)

# d2F/dt2
d2Fx_dt2 = I * kx * (dExx_dt + dExy_dt + dExz_dt)
d2Fy_dt2 = I * ky * (dEyx_dt + dEyy_dt + dEyz_dt)
d2Fz_dt2 = I * kz * (dEzx_dt + dEzy_dt + dEzz_dt)
d2F_dt2 = Matrix([[d2Fx_dt2], [d2Fy_dt2], [d2Fz_dt2]])

# d2G/dt2
d2Gx_dt2 = c**2 * I * kx * (dBxx_dt + dBxy_dt + dBxz_dt)
d2Gy_dt2 = c**2 * I * ky * (dByx_dt + dByy_dt + dByz_dt)
d2Gz_dt2 = c**2 * I * kz * (dBzx_dt + dBzy_dt + dBzz_dt)
d2G_dt2 = Matrix([[d2Gx_dt2], [d2Gy_dt2], [d2Gz_dt2]])

for i in range(dd):
    d2E_dt2[i] = sp.expand(d2E_dt2[i])

for i in range(dd):
    d2B_dt2[i] = sp.expand(d2B_dt2[i])

for i in range(3):
    d2F_dt2[i] = sp.expand(d2F_dt2[i])

for i in range(3):
    d2G_dt2[i] = sp.expand(d2G_dt2[i])

# Extended array for E and G
EG = zeros(DD, 1)
for i in range(dd):
    EG[i] = E[i]
for i in range(dd, DD):
    EG[i] = G[i - dd]

# dEG/dt
dEG_dt = zeros(DD, 1)
for i in range(dd):
    dEG_dt[i] = dE_dt[i]
for i in range(dd, DD):
    dEG_dt[i] = dG_dt[i - dd]

# d2EG/dt2
d2EG_dt2 = zeros(DD, 1)
for i in range(dd):
    d2EG_dt2[i] = d2E_dt2[i]
for i in range(dd, DD):
    d2EG_dt2[i] = d2G_dt2[i - dd]

# Extended array for B and F
BF = zeros(DD, 1)
for i in range(dd):
    BF[i] = B[i]
for i in range(dd, DD):
    BF[i] = F[i - dd]

# dBF/dt
dBF_dt = zeros(DD, 1)
for i in range(dd):
    dBF_dt[i] = dB_dt[i]
for i in range(dd, DD):
    dBF_dt[i] = dF_dt[i - dd]

# d2BF/dt2
d2BF_dt2 = zeros(DD, 1)
for i in range(dd):
    d2BF_dt2[i] = d2B_dt2[i]
for i in range(dd, DD):
    d2BF_dt2[i] = d2F_dt2[i - dd]

### Solve second-order ODEs for $\boldsymbol{E}$, $\boldsymbol{B}$, $F$ and $G$:

In [ ]:
print(r"Solve equations for E and G:")
EG_t, EG_new = evolve(EG, dEG_dt, d2EG_dt2)

print(r"Solve equations for B and F:")
BF_t, BF_new = evolve(BF, dBF_dt, d2BF_dt2)

# Check correctness by taking *first* derivative
# and comparing with initial right-hand side at time tn
# E,G
diff = EG_t.diff(t).subs(t, tn).subs(om, c * knorm).expand() - dEG_dt
diff.simplify()
if diff != zeros(DD, 1):
    print("Integration in time failed")
    display(diff)
# B,F
diff = BF_t.diff(t).subs(t, tn).subs(om, c * knorm).expand() - dBF_dt
diff.simplify()
if diff != zeros(DD, 1):
    print("Integration in time failed")
    display(diff)

### Coefficients of PSATD equations in PML:

In [ ]:
# Code generation

#       0,   1,   2,   3,   4,   5,   6,   7,   8,  9, 10, 11
# EG: Exx, Exy, Exz, Eyx, Eyy, Eyz, Ezx, Ezy, Ezz, Gx, Gy, Gz
# BF: Bxx, Bxy, Bxz, Byx, Byy, Byz, Bzx, Bzy, Bzz, Fx, Fy, Fz

# Select update equation (left hand side)
X_new = BF_new[0]

# Extract individual terms (right hand side)
for i in range(DD):
    X = EG[i]
    C1 = X_new.coeff(X, 1).simplify()
    print(r"Coefficient multiplying " + str(X))
    display(C1)
    # print(ccode(Assignment(sp.symbols(r'LHS'), C1)))
for i in range(DD):
    X = BF[i]
    C2 = X_new.coeff(X, 1).simplify()
    print(r"Coefficient multiplying " + str(X))
    display(C2)
    # print(ccode(Assignment(sp.symbols(r'LHS'), C2)))